In [1]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

This notebook is used to test the conversion of non-deterministic <span style="font-variant:small-caps;">Fsm</span>s into deterministic <span style="font-variant:small-caps;">Fsm</span>s.

In [2]:
import { instance } from "@viz-js/viz";
import { RecursiveSet, RecursiveMap, Tuple } from "./recursive-set";
const viz = await instance();

In order to represent the <span style="font-variant:small-caps;">Fsm</span>s graphically, we use the notebook `FSM-2-Dot.ipynb`.  This notebook uses the package `graphviz`.

In [3]:
import {
    NFA,
    DFA,
    nfa2dfa,
    TransRel,
    State,
    DFAState,
    Char
} from "./01-NFA-2-DFA";
import { nfa2dot, dfa2dot } from "./FSM-2-Dot";

In [4]:
const stateList = Array.from({ length: 8 }, (_, i) => `q${i}`);
const States = new RecursiveSet<State>(...stateList);
States;

{q0, q1, q2, q3, q4, q5, q6, q7}


In [5]:
const Sigma = new RecursiveSet<Char>("a", "b");

In [6]:
const delta: TransRel = new RecursiveMap();
delta.set(new Tuple("q0", "ε"), new RecursiveSet("q1", "q2"));
delta.set(new Tuple("q5", "ε"), new RecursiveSet("q7"));
delta.set(new Tuple("q6", "ε"), new RecursiveSet("q7"));
delta.set(new Tuple("q7", "ε"), new RecursiveSet("q0"));
delta.set(new Tuple("q1", "b"), new RecursiveSet("q3"));
delta.set(new Tuple("q2", "a"), new RecursiveSet("q4"));
delta.set(new Tuple("q3", "a"), new RecursiveSet("q5"));
delta.set(new Tuple("q4", "b"), new RecursiveSet("q6"));
delta;

RecursiveMap(8) {
  (q0, ε) => {q1, q2},
  (q1, b) => {q3},
  (q2, a) => {q4},
  (q3, a) => {q5},
  (q4, b) => {q6},
  (q5, ε) => {q7},
  (q6, ε) => {q7},
  (q7, ε) => {q0}
}


The non-deterministic <span style="font-variant:small-caps;">Fsm</span> defined below is taken from the lecture notes.

In [17]:
const nfa44: NFA = {
    Q: States,
    Σ: Sigma,
    δ: delta,
    q0: "q0",
    A: new RecursiveSet("q7"),
};
nfa44;

{
  Q: {q0, q1, q2, q3, q4, q5, q6, q7},
  'Σ': {a, b},
  'δ': RecursiveMap(8) {
    (q0, ε) => {q1, q2},
    (q1, b) => {q3},
    (q2, a) => {q4},
    (q3, a) => {q5},
    (q4, b) => {q6},
    (q5, ε) => {q7},
    (q6, ε) => {q7},
    (q7, ε) => {q0}
  },
  q0: 'q0',
  A: {q7}
}


The function `nfa2dot`can be used to render this <span style="font-variant:small-caps;">Fsm</span>.

In [18]:
const dotNFA: string = nfa2dot(nfa44);

In [19]:
display.html(viz.renderString(dotNFA, { format: "svg" }));

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.1 (20251213.1925)
 -->
<!-- Title: Non-Deterministic FSM Pages: 1 -->
 
 
 Non-Deterministic FSM 
 
<!-- start_ghost -->
 
 start_ghost 
 
<!-- q0 -->
 
 q0 
 
 q0 
 
<!-- start_ghost->q0 -->
 
 start_ghost->q0 
 
 
 
<!-- q1 -->
 
 q1 
 
 q1 
 
<!-- q0->q1 -->
 
 q0->q1 
 
 
 ε 
 
<!-- q2 -->
 
 q2 
 
 q2 
 
<!-- q0->q2 -->
 
 q0->q2 
 
 
 ε 
 
<!-- q3 -->
 
 q3 
 
 q3 
 
<!-- q1->q3 -->
 
 q1->q3 
 
 
 b 
 
<!-- q4 -->
 
 q4 
 
 q4 
 
<!-- q2->q4 -->
 
 q2->q4 
 
 
 a 
 
<!-- q5 -->
 
 q5 
 
 q5 
 
<!-- q3->q5 -->
 
 q3->q5 
 
 
 a 
 
<!-- q6 -->
 
 q6 
 
 q6 
 
<!-- q4->q6 -->
 
 q4->q6 
 
 
 b 
 
<!-- q7 -->
 
 q7 
 
 
 q7 
 
<!-- q5->q7 -->
 
 q5->q7 
 
 
 ε 
 
<!-- q6->q7 -->
 
 q6->q7 
 
 
 ε 
 
<!-- q7->q0 -->
 
 q7->q0 
 
 
 ε

This recognizes the same language as the language described by
$$ (a \cdot b + b \cdot a) \cdot (a \cdot b + b \cdot a)^* $$

Let us convert it into a deterministic Fsm:

In [22]:
const dfa44: DFA = nfa2dfa(nfa44);
dfa44;

{
  Q: {{q4}, {q0, q1, q2, q5, q7}, {q0, q1, q2, q6, q7}, {q0, q1, q2}, {q3}, {}},
  'Σ': {a, b},
  'δ': RecursiveMap(12) {
    ({}, a) => {},
    ({}, b) => {},
    ({q0, q1, q2, q5, q7}, a) => {q4},
    ({q0, q1, q2, q5, q7}, b) => {q3},
    ({q0, q1, q2, q6, q7}, a) => {q4},
    ({q0, q1, q2, q6, q7}, b) => {q3},
    ({q0, q1, q2}, a) => {q4},
    ({q0, q1, q2}, b) => {q3},
    ({q3}, a) => {q0, q1, q2, q5, q7},
    ({q3}, b) => {},
    ({q4}, a) => {},
    ({q4}, b) => {q0, q1, q2, q6, q7}
  },
  q0: {q0, q1, q2},
  A: {{q0, q1, q2, q5, q7}, {q0, q1, q2, q6, q7}}
}


The function `dfa2dot`can be used to render this <span style="font-variant:small-caps;">Fsm</span>.

In [21]:
const dotDFA = dfa2dot(dfa44);

In [23]:
display.html(viz.renderString(dotDFA, { format: "svg" }));

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.1 (20251213.1925)
 -->
<!-- Title: Deterministic FSM Pages: 1 -->
 
 
 Deterministic FSM 
 
<!-- start_ghost -->
 
 start_ghost 
 
<!-- S0 -->
 
 S0 
 
 {q0, q1, q2} 
 
<!-- start_ghost->S0 -->
 
 start_ghost->S0 
 
 
 
<!-- S1 -->
 
 S1 
 
 {q4} 
 
<!-- S0->S1 -->
 
 S0->S1 
 
 
 a 
 
<!-- S2 -->
 
 S2 
 
 {q3} 
 
<!-- S0->S2 -->
 
 S0->S2 
 
 
 b 
 
<!-- S3 -->
 
 S3 
 
 {} 
 
<!-- S1->S3 -->
 
 S1->S3 
 
 
 a 
 
<!-- S4 -->
 
 S4 
 
 
 {q0, q1, q2, q6, q7} 
 
<!-- S1->S4 -->
 
 S1->S4 
 
 
 b 
 
<!-- S2->S3 -->
 
 S2->S3 
 
 
 b 
 
<!-- S5 -->
 
 S5 
 
 
 {q0, q1, q2, q5, q7} 
 
<!-- S2->S5 -->
 
 S2->S5 
 
 
 a 
 
<!-- S3->S3 -->
 
 S3->S3 
 
 
 a 
 
<!-- S3->S3 -->
 
 S3->S3 
 
 
 b 
 
<!-- S4->S1 -->
 
 S4->S1 
 
 
 a 
 
<!-- S4->S2 -->
 
 S4->S2 
 
 
 b 
 
<!-- S5->S1 -->
 
 S5->S1 
 
 
 a 
 
<!-- S5->S2 -->
 
 S5->S2 
 
 
 b